<a href="https://colab.research.google.com/github/natdebandi/hate_speech_ar/blob/main/3_GPT_finetuning_multiclass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP final - reconocimiento de discursos discriminatorios en Twitter
## (3 )Finetuning de GPT 4 mini para clasificación binaria

**Natalia Dedandi**



La documentación de la API se encuentra aqui:

https://platform.openai.com/docs/api-reference/introduction


Guia para finetuning:

https://medium.com/@garethcull/fine-tuning-ai-models-a-practical-guide-for-beginners-dc313b2e0f76

In [1]:
pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.9/335.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.5 MB/s eta 0:00:00


In [2]:
!pip install datasets seaborn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
      S

In [3]:
import json
from openai import OpenAI
import os
from google.colab import userdata
#seteo la KEY
os.environ['OPENAI_API_KEY'] = userdata.get('openIA_key')




In [4]:
#creo el cliente OPENAI con mi usuario y proyecto
client = OpenAI(
  organization='org-1uHjwiaB3OlPzoxfVzhqOSzs',
  project='proj_2fII7izwVGgYaERNNKhhMx4l',
)

Levanto los datos de entrenamiento

In [5]:
from datasets import load_dataset
import pandas as pd

ds1 = load_dataset("piuba-bigdata/contextualized_hate_speech")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/36420 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11343 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/9106 [00:00<?, ? examples/s]

Tengo que transformar el DS de train en un conjunto de mensajes para pasarle a GPT para finetuning.

In [20]:
# prompt: Create a dataframe from ds1 with id text and hateful cols

df_train = pd.DataFrame(ds1['train'])[['id', 'text', 'HATEFUL', 'body', 'CALLS', 'WOMEN', 'LGBTI', 'RACISM', 'CLASS', 'POLITICS', 'DISABLED', 'APPEARANCE', 'CRIMINAL']]


In [22]:
# prompt: create a datfram from ds1 for train
df_train[0:10]

,id,text,HATEFUL,body,CALLS,WOMEN,LGBTI,RACISM,CLASS,POLITICS,...,CRIMINAL,CALLS_label,WOMEN_label,LGBTI_label,RACISM_label,CLASS_label,POLITICS_label,DISABLED_label,APPEARANCE_label,CRIMINAL_label
0,343726,@usuario Uno menos,1,"Un hombre de 46 años, que cumplía una condena ...",0,0,0,0,0,0,...,1,,,,,,,,,CRIMINAL
1,384799,@usuario #QueSeVayanTodos @usuario @usuario @u...,0,"Solange Musse, la hija del hombre que había vi...",0,0,0,0,0,0,...,0,,,,,,,,,
2,399435,"@usuario Te falta negociar con Cuba, mamerto.",0,Alberto Fernández desplegó una estrategia de a...,0,0,0,0,0,0,...,0,,,,,,,,,
3,403171,@usuario Se prepara para las milicias armadas,0,"El ex presidente de Bolivia, Evo Morales, part...",0,0,0,0,0,0,...,0,,,,,,,,,
4,394186,@usuario Seguimos alimentando vagos,1,"Una tarde lluviosa de marzo, cuando la pandemi...",0,0,0,0,1,0,...,0,,,,,CLASS,,,,
5,345041,@usuario Mmmmm,0,"Una vacuna, del laboratorio Sinopharm, es desa...",0,0,0,0,0,0,...,0,,,,,,,,,
6,339268,@usuario ¿Copiando la receta de la sopa de mur...,0,En un informe difundido durante las últimas ho...,0,0,0,0,0,0,...,0,,,,,,,,,
7,360180,@usuario lastima que no murio... necesitamos u...,1,En un extraño episodio que todavía se investig...,0,0,0,0,0,1,...,0,,,,,,POLITICS,,,
8,388977,@usuario Un Grande Gildo.......,0,"Zunilda Gómez, de Clorinda, perdió su embarazo...",0,0,0,0,0,0,...,0,,,,,,,,,
9,374959,"@usuario No sé olvide Santiago Cafiero ,cuando...",0,"\n\n\nEl Jefe de Gabinete, Santiago Cafiero, c...",0,0,0,0,0,0,...,0,,,,,,,,,


In [21]:
# ajustar los valores y crear una variable binaria entera 1 y 0
df_train['CALLS_label'] = df_train['CALLS'].apply(lambda x: 'CALLS' if  x==1 else "" )
df_train['WOMEN_label'] = df_train['WOMEN'].apply(lambda x: 'WOMEN' if  x==1 else "")
df_train['LGBTI_label'] = df_train['LGBTI'].apply(lambda x: 'LGBTI' if  x==1 else "")
df_train['RACISM_label'] = df_train['RACISM'].apply(lambda x: 'RACISM' if x==1 else "" )
df_train['CLASS_label'] = df_train['CLASS'].apply(lambda x: 'CLASS' if  x==1  else "")
df_train['POLITICS_label'] = df_train['POLITICS'].apply(lambda x: 'POLITICS' if  x==1  else "")
df_train['DISABLED_label'] = df_train['DISABLED'].apply(lambda x: 'DISABLED' if  x==1  else "")
df_train['APPEARANCE_label'] = df_train['APPEARANCE'].apply(lambda x: 'APPEARANCE' if  x==1  else "")
df_train['CRIMINAL_label'] = df_train['CRIMINAL'].apply(lambda x: 'CRIMINAL' if  x==1  else "")

# Create a new column by merging non-empty "_label" columns
df_train['predicted_labels'] = df_train.filter(like='_label').apply(lambda x: ', '.join(x[x != '']), axis=1)

In [24]:

df_train[0:5]


,id,text,HATEFUL,body,CALLS,WOMEN,LGBTI,RACISM,CLASS,POLITICS,...,CALLS_label,WOMEN_label,LGBTI_label,RACISM_label,CLASS_label,POLITICS_label,DISABLED_label,APPEARANCE_label,CRIMINAL_label,predicted_labels
0,343726,@usuario Uno menos,1,"Un hombre de 46 años, que cumplía una condena ...",0,0,0,0,0,0,...,,,,,,,,,CRIMINAL,CRIMINAL
1,384799,@usuario #QueSeVayanTodos @usuario @usuario @u...,0,"Solange Musse, la hija del hombre que había vi...",0,0,0,0,0,0,...,,,,,,,,,,
2,399435,"@usuario Te falta negociar con Cuba, mamerto.",0,Alberto Fernández desplegó una estrategia de a...,0,0,0,0,0,0,...,,,,,,,,,,
3,403171,@usuario Se prepara para las milicias armadas,0,"El ex presidente de Bolivia, Evo Morales, part...",0,0,0,0,0,0,...,,,,,,,,,,
4,394186,@usuario Seguimos alimentando vagos,1,"Una tarde lluviosa de marzo, cuando la pandemi...",0,0,0,0,1,0,...,,,,,CLASS,,,,,CLASS


Creo una lista BATCH con el formato de mensajes que debo enviar a GPT para finetunning

In [25]:


  # Convert the dataframe into a list of dictionaries with custom_id and messages
  batch = [
      {'messages': [
                  {'role': 'system', 'content': 'You are trained to analyze and detect the sentiment of the given text.You must classify a text with one or more of the following labels: WOMEN when there is discrimination against women, LGBTI when there is discrimination against gays, lesbians or transsexuals, RACISM when there is discrimination against people according to their skin color, CLASS  when there is discrimination for being poor or living in a marginal or poor place, POLITICS is hatred of politicians, DISABLED when there is discrimination against people with disabilities, APPEARENCE when there is discrimination against people because of their physical appearance, specifically fatphobia, CRIMINAL when there is discrimination against people because of a criminal record or links to criminality and  CALLS is hateful text that incites violence. If the text is not hateful indicate NOT HATEFUL'},
                  {'role': 'user', 'content': row['text']},
                  {'role': 'assistant', 'content': row['predicted_labels']}
                  ]
       }
      for _, row in df_train.iterrows()
  ]


In [26]:
batch[0]

{'messages': [{'role': 'system',
   'content': 'You are trained to analyze and detect the sentiment of the given text.You must classify a text with one or more of the following labels: WOMEN when there is discrimination against women, LGBTI when there is discrimination against gays, lesbians or transsexuals, RACISM when there is discrimination against people according to their skin color, CLASS  when there is discrimination for being poor or living in a marginal or poor place, POLITICS is hatred of politicians, DISABLED when there is discrimination against people with disabilities, APPEARENCE when there is discrimination against people because of their physical appearance, specifically fatphobia, CRIMINAL when there is discrimination against people because of a criminal record or links to criminality and  CALLS is hateful text that incites violence. If the text is not hateful indicate NOT HATEFUL'},
  {'role': 'user', 'content': '@usuario Uno menos'},
  {'role': 'assistant', 'content':

In [ ]:
# Save the batch to a .jsonl file
with open('batch_gpt_bin_train.jsonl', 'w') as f:
    for entry in batch:
        json.dump(entry, f)
        f.write('\n')

print("Batch file created successfully in .jsonl format!")

Batch file created successfully in .jsonl format!


Ya tengo creado el archivo y lo subo para finetunning

In [ ]:
from openai import OpenAI
client = OpenAI()

client.files.create(
  file=open("batch_gpt_bin_train.jsonl", "rb"),
  purpose="fine-tune"
)

FileObject(id='file-I6srp0hpoIQllyxQRQ4U24iB', bytes=39613877, created_at=1722719183, filename='batch_gpt_bin_train.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [ ]:
client.fine_tuning.jobs.create(
  training_file="file-I6srp0hpoIQllyxQRQ4U24iB",
  model="gpt-3.5-turbo-0125",
  hyperparameters={
    "n_epochs":5
  },
  suffix="hate_speech_v2"
)

FineTuningJob(id='ftjob-3hLvVaaM5T51D19lXitFUC5Q', created_at=1722719196, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=5, batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-1uHjwiaB3OlPzoxfVzhqOSzs', result_files=[], seed=1064557800, status='validating_files', trained_tokens=None, training_file='file-I6srp0hpoIQllyxQRQ4U24iB', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix='hate_speech_v2')

In [ ]:
fine_tuning_job = client.fine_tuning.jobs.retrieve(
  fine_tuning_job_id="ftjob-3hLvVaaM5T51D19lXitFUC5Q"
)

print(fine_tuning_job)


FineTuningJob(id='ftjob-3hLvVaaM5T51D19lXitFUC5Q', created_at=1722719196, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=5, batch_size=121, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-1uHjwiaB3OlPzoxfVzhqOSzs', result_files=[], seed=1064557800, status='validating_files', trained_tokens=None, training_file='file-I6srp0hpoIQllyxQRQ4U24iB', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix='hate_speech_v2')
